In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *
import pyspark.sql.functions as f



In [0]:
def flatten_check(main_df):
    exist = True
    while exist:
        for col_name, col_type in main_df.dtypes:
            # This will take care of non array types i.e (STRUCT STRING)
            if col_type[:6] != 'array':
                main_df = flatten_df(main_df)

            # This will take care of non struct types i.e (ARRAY)
            if col_type[:5] == 'array':
                main_df = exploade_array(main_df, col_name)

            struct_type = [c[0] for c in main_df.dtypes if c[1][:6] == 'struct']
            array_type = [c[0] for c in main_df.dtypes if c[1][:5] == 'array']

            if len(struct_type) > 0 or len(array_type) > 0:
                exist = True
            else:
                exist = False

    return main_df


def flatten_df(nested_df):

    flat_col = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
    nested_col = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

    if len(nested_col) >0:
        flat_df = nested_df.select(flat_col + [col(c+'.'+nc).alias(c+'_'+nc) for c in nested_col for nc in nested_df.select(c+'.*').columns])
    else:
        flat_df = nested_df
        
    return flat_df
            


def exploade_array(main_df, new_col):

    explode_col_name = main_df.select(new_col)

    explode_df = main_df.withColumn(new_col, explode_outer(explode_col_name[0]))

    return explode_df



            


In [0]:
def fetch_max_count(column, table_name):
    try:
        query = "SELECT CASE WHEN MaxID = -1 THEN 0 ELSE MaxID END AS MaxID FROM  (SELECT nvl(MAX({0}),0) AS MaxID FROM {1})".format(column, table_name)
        max_count_df = spark.sql(query)
        max_count_df.createOrReplaceTempView("MaxID")
        return max_count_df

    except Exception as e:
        error = f"{e}"
        print("Error "+ error)

